# Early Stopping
The following example shows how to use early stopping while training a model and how to create checkpoints.

In [2]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [4]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [6]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=1, 
                                     k=10,
                                     scoring_type='TransE')


# compile the model with loss and optimizer
model.compile(optimizer='adam', loss='multiclass_nll')

# Use this for checkpoints at regular intervals
#checkpoint = tf.keras.callbacks.ModelCheckpoint('./chkpt_transe', monitor='val_mrr', verbose=1, save_best_only=True, mode='min')

# Use this for early stopping
checkpoint = tf.keras.callbacks.EarlyStopping(monitor="val_mrr",            # which metrics to monitor
                                              patience=3,                   # If the monitored metric doesnt improve 
                                                                            # for these many checks the model early stops
                                              verbose=1,                    # verbosity
                                              mode="max",                   # how to compare the monitored metrics. 
                                                                            # max - means higher is better
                                              restore_best_weights=True)    # restore the weights with best value

dataset = load_fb15k_237()

model.fit(dataset['train'],
          batch_size=10000,
          epochs=100,
          validation_freq=2,                            # Epochs to elapse before next evaluation
          validation_batch_size=100,                    
          validation_burn_in=1,                         # Epoch to start the validation process
          validation_data = dataset['valid'][::100],
          callbacks=[checkpoint])                       # Pass the callback to the fit function


Epoch 1/100
29/29 [==============================] - 3s 94ms/step - loss: 6622.9019
Epoch 2/100
29/29 [==============================] - ETA: 0s - loss: 6493.6064

2023-02-08 13:21:22.984591: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 2s 81ms/step - loss: 6493.6064 - val_mrr: 0.0684 - val_mr: 3258.6080 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1790 - val_hits@100: 0.3466
Epoch 3/100
29/29 [==============================] - 1s 34ms/step - loss: 6365.1802
Epoch 4/100
29/29 [==============================] - ETA: 0s - loss: 6234.0171

2023-02-08 13:21:26.191442: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 2s 69ms/step - loss: 6234.0171 - val_mrr: 0.0809 - val_mr: 1924.0540 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2102 - val_hits@100: 0.4006
Epoch 5/100
29/29 [==============================] - 2s 59ms/step - loss: 6101.5239
Epoch 6/100
26/29 [=========================>....] - ETA: 0s - loss: 6003.9404

...

29/29 [==============================] - 2s 69ms/step - loss: 4488.2705 - val_mrr: 0.0833 - val_mr: 755.6392 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2216 - val_hits@100: 0.5199
Epoch 19/100
29/29 [==============================] - 1s 29ms/step - loss: 4393.5967
Epoch 20/100
29/29 [==============================] - ETA: 0s - loss: 4303.0571

2023-02-08 13:21:47.841319: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 1s 48ms/step - loss: 4303.0571 - val_mrr: 0.0827 - val_mr: 739.2500 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2216 - val_hits@100: 0.5227
Epoch 21/100
29/29 [==============================] - 0s 17ms/step - loss: 4217.0073
Epoch 22/100
      2/Unknown - 0s 152ms/step===>..] - ETA: 0s - loss: 4140.2354

2023-02-08 13:21:50.470017: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


29/29 [==============================] - 2s 64ms/step - loss: 4134.3545 - val_mrr: 0.0839 - val_mr: 737.0227 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2244 - val_hits@100: 0.5170
Restoring model weights from the end of the best epoch: 16.
Epoch 22: early stopping


In [7]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 616s 3s/step
MR: 600.0142626480086
MRR: 0.18888812773966743
hits@1: 0.1267491926803014
hits@10: 0.3130687934240141


In [ ]:
# if using ModelCheckpoint then we can restore the checkpoints using restore model
# from ampligraph.utils import restore_model
# model = restore_model('chkpt_transe')